## Hugging_Face Embeddings

In [2]:
from langchain_huggingface import HuggingFaceEmbeddings

In [3]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
embeddings

/home/hammadali08/PycharmProjects/RAG/.venv/lib64/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [4]:
vectors=embeddings.embed_query("Hello I am Hammad Ali Tahir")
print(vectors)
print(len(vectors))

[0.008959689177572727, 0.09178251028060913, -0.0042881350964307785, 0.05720607936382294, -0.07849406450986862, -0.05122251436114311, 0.16765734553337097, -0.05901440978050232, 0.07593921571969986, -0.010059552267193794, 0.08655273169279099, -0.09947802871465683, 0.021933089941740036, -0.06579573452472687, 0.07003957033157349, -0.05746196582913399, 0.020773770287632942, -0.02633540891110897, -0.03527873381972313, -0.0733388140797615, -0.11658710241317749, -0.03513731807470322, 0.006203813012689352, 0.032315172255039215, -0.11720556020736694, -0.025680026039481163, 0.006114559713751078, 0.09403189271688461, 0.05834360793232918, -0.06230120733380318, -0.017115095630288124, 0.03678779676556587, 0.07167858630418777, -0.012926035560667515, -0.012077664956450462, 0.12244991958141327, -0.01254356186836958, -0.016596350818872452, 0.014732501469552517, 0.030170902609825134, 0.028035953640937805, -0.041900791227817535, 0.006529353559017181, -0.06054788827896118, 0.08014234900474548, -0.0735695436

### Applying over real world pdf

In [21]:
from typing import List, Dict, Optional
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

class SmartPDFProcessor:
    """Advanced PDF processing with error handling + custom metadata injection"""

    def __init__(self, chunk_size: int = 2000, chunk_overlap: int = 100):
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            separators=[" "],
        )

    def process_pdf(
        self, 
        pdf_path: str, 
        custom_metadata: Optional[Dict[str, str]] = None
    ) -> List[Document]:
        """Process PDF with smart chunking and metadata enhancement"""

        # Load PDF
        loader = PyPDFLoader(pdf_path)
        pages = loader.load()

        processed_chunks = []

        for page_num, page in enumerate(pages):
            cleaned_text = self._clean_text(page.page_content)

            # Skip nearly empty pages
            if len(cleaned_text.strip()) < 50:
                continue

            # Step 1: Page details FIRST
            metadata = {
                "page": page_num + 1,
                "total_pages": len(pages),
                "chunk_method": "smart_pdf_processor",
                "char_count": len(cleaned_text),
            }

            # Step 2: Add PDF metadata (if any)
            if page.metadata:
                metadata.update(page.metadata)

            # Step 3: Add custom metadata (overrides everything if same key)
            if custom_metadata:
                metadata.update(custom_metadata)

            # Create chunks
            chunks = self.text_splitter.create_documents(
                texts=[cleaned_text],
                metadatas=[metadata]
            )

            processed_chunks.extend(chunks)

        return processed_chunks

    def _clean_text(self, text: str) -> str:
    # Remove excessive whitespace
        text = " ".join(text.split())
    
    # Fix common PDF extraction issues
        text = text.replace("ﬁ", "fi")
        text = text.replace("ﬂ", "fl")
    
        return text


In [22]:
processed_data=SmartPDFProcessor()
processed_chunks=processed_data.process_pdf("/home/hammadali08/Personal/FYP Datasets/Pakistan-A-Hard-Country-by-Anatol-Lieven-.pdf")

In [23]:
Book_embeddings=embeddings.embed_documents([doc.page_content for doc in processed_chunks])

In [24]:
print(Book_embeddings[1000])

[-0.04111848399043083, -0.013499563559889793, 0.002636983525007963, -0.07387751340866089, -0.0560770183801651, 0.06765451282262802, 0.07907240837812424, 0.06737985461950302, -0.019272392615675926, -0.028592867776751518, 0.016514457762241364, 0.06649746745824814, 0.042857881635427475, -0.010185573250055313, -0.020621126517653465, 0.025265797972679138, 0.007741949055343866, 0.0746905729174614, -0.07470844686031342, -0.050839994102716446, 0.017977861687541008, 0.04661732167005539, -0.03982268273830414, -0.03255411610007286, -0.02455032616853714, 0.044678281992673874, -0.031913526356220245, -0.04881593585014343, -0.024980470538139343, -0.08706148713827133, 0.08939305692911148, 0.05214192718267441, 0.02190963178873062, -0.03127215430140495, -0.018406372517347336, 0.016821980476379395, 0.010184616781771183, 0.01702995039522648, -0.01485525257885456, 0.035541631281375885, 0.045560527592897415, -0.019745446741580963, 0.04007119685411453, 0.020239394158124924, 0.08284152299165726, 0.02426203526

### Cosine Similarity:

In [25]:
import numpy as np
def cosine_similarity(vec1, vec2):
    """Compute cosine similarity between two vectors."""
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    if norm_vec1 == 0 or norm_vec2 == 0:
        return 0.0
    return dot_product / (norm_vec1 * norm_vec2)

### Semantic Similiarity

In [26]:
def semantic_search(query: str, embeddings_list: List[List[float]], top_k: int = 5) -> List[int]:
    """Perform semantic search to find top_k similar documents."""
    query_embedding = embeddings.embed_query(query)
    similarities = [cosine_similarity(query_embedding, doc_embedding) for doc_embedding in embeddings_list]
    top_indices = np.argsort(similarities)[-top_k:][::-1]
    return top_indices.tolist()

In [31]:
data=semantic_search("Punjab",Book_embeddings,top_k=2)

In [32]:
data

[1129, 598]

In [33]:
print(processed_chunks[1129])

page_content='Pakistan A Hard Country by Anatol Lieven 433 B ap s i S i d hw a ( ed . ) ,Beloved City: Writings on Lahore ( O xfo r d Un i v er s i t y P r es s ,Kar ac hi , 2005) Gur har p al S i n g h an d I an Talbo t ,Punjabi Identity: Continuity and Change ( M an o har P ubli s her s ,N ew D elhi ,1996 ) I an Talbo tan d S hi n d er Than d i ,People on the Move: Punjabi Colonial and Post-Colonial Migration ( O xfo r dUn i v er s i t y P r es s ,O xfo r d ,2 004) Tan TaiYo n g ,The Garrison State: The Military, Government and Society in Colonial Punjab, 1849–1947( V an g uar dB o o k s ,L aho r e,2005 ) CHAPTER 8 SINDH F er o z A hm ed ,‘Ethnicity and Politics: The Rise of Muhajir Separatism ’ ,S o ut h A s i a B ullet i n , 8 ( 1988) H am za A lav i , ‘Nationhood and the Nationalities in Pakistan’, i n H as t i n g s D o n n an an d P n i n a W er bn er ( ed s ) ,Economy and Culture in Pakistan: Migrants and Cities in a Muslim Society ( M ac m i llan ,L o n d o n ,1991 ) S ar ah A